<a href="https://colab.research.google.com/github/rajivsam/window_size_experiments/blob/main/notebooks/window_size_experiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/rajivsam/window_size_experiments/

Cloning into 'window_size_experiments'...
remote: Enumerating objects: 29, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 29 (delta 4), reused 25 (delta 3), pack-reused 0 (from 0)
Receiving objects: 100% (29/29), 1.88 MiB | 9.66 MiB/s, done.
Resolving deltas: 100% (4/4), done.


In [5]:
fp = "/content/window_size_experiments/data/data_phase2/001_UCR_Anomaly_35000.txt"

In [6]:
!pip install -r /content/window_size_experiments/requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.5/176.5 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 17.4 MB/s eta 0:00:00


In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import os

import plotly.graph_objects as go

In [8]:
movingAvgResiduals = []
window_sizes = []

In [9]:
def movmean(ts, w):
    """
    # faster solution of moving ave
    moving_avg = np.cumsum(ts, dtype=float)
    moving_avg[w:] = moving_avg[w:] - moving_avg[:-w]
    return moving_avg[w-1:] / w
    """


    moving_avg = np.cumsum(ts, dtype=float)
    moving_avg[w:] = moving_avg[w:] - moving_avg[:-w]
    return moving_avg[w-1:] / w



def original_finding_window_size(ts, start, end, step):
    """
    finidng appropriate window size using movemean
    """
    threshold = float('inf')
    all_averages = []

    #step size is now parametrized
    for w in range(start, end, step):

        movingAvg = np.array(movmean(ts, w))

        all_averages.append(movingAvg)
        window_sizes.append(w)



    for i, w in enumerate(window_sizes):
        moving_avg = all_averages[i][:len(all_averages[-1])]

        movingAvgResidual = np.log(abs(moving_avg - (moving_avg).mean()).sum())

        movingAvgResiduals.append(movingAvgResidual)

    b = (np.diff(np.sign(np.diff(movingAvgResiduals))) > 0).nonzero()[0] + 1  #local min , RS: This identifies the local minima
    reswin = []
    try:
        for i in range(3):
            reswin.append(window_sizes[b[i]]/ (i+1))
        reswin = np.array(reswin)
        winTime = 0.8 * reswin[0] + 0.15 * reswin[1] + 0.05 * reswin[2] # RS: This seems to be some weighting of the first 3 local minima
#         winTime = np.mean(reswin)
        conf = np.std(reswin)/  np.sqrt(3)
    except:
        winTime = window_sizes[b[0]] # RS: If you don't have 3 minima, default to the first, the first is a good estimate anyway.
        conf = np.nan
#     kn =  windowLocator(window_sizes, movingAvgResiduals) # window size (the reason I choose kn is because when I wanted to be the same name as kneelocator function)
#     movingAvgResiduals = list(np.array(movingAvgResiduals[:-1]) - np.array(movingAvgResiduals[1:]))

    return winTime, conf, reswin

In [10]:
fp = "/content/window_size_experiments/data/data_phase2/001_UCR_Anomaly_35000.txt"
df = pd.read_csv(fp)
ts = df.values

In [11]:
# using s and e values in the listing for ALgorithm 1 in the paper.
start = 10
end = 1000
step = 10
winTime, conf, reswin= original_finding_window_size(ts, start, end, step)

In [12]:
dfp = pd.DataFrame.from_records({"window_size": window_sizes, "moving_dist": movingAvgResiduals})

In [13]:
fig = px.line(dfp, x="window_size", y="moving_dist", title='Moving Dist vs Window Size',  symbol='moving_dist')
fig.show()

In [14]:
fp = "/content/window_size_experiments/data/data_phase2/007_UCR_Anomaly_4000.txt"
df = pd.read_csv(fp)
ts = df.values

In [15]:
movingAvgResiduals = []
window_sizes = []
# using s and e values in the listing for ALgorithm 1 in the paper.
start = 10
end = 100
step = 1
winTime, conf, reswin= original_finding_window_size(ts, start, end, step)

In [16]:
dfp = pd.DataFrame.from_records({"window_size": window_sizes, "moving_dist": movingAvgResiduals})
fig = px.line(dfp, x="window_size", y="moving_dist", title='Moving Dist vs Window Size',  symbol='moving_dist')
fig.show()